# Using QUIP potentials with Atomic Simulation Environment

- quippy `Potential` objects can be used as ASE calculators
- Communicate with other packages using ASE as *lingua franca*

## Example: vacancy formation energy

- Generate structure with `ASE` lattice tools
- Stillinger-Weber potential implementation from `QUIP`
- Elastic constant fitting routine from `matscipy`, internal relaxations with `ASE` FIRE minimiser

In [5]:
from ase.build import bulk
from ase.optimize import LBFGS
from quippy.potential import Potential

si = bulk('Si', a=5.44, cubic=True)
sw_pot = Potential('IP SW', param_str="""
<SW_params n_types="3" label="PRB_31_plus_H_Ge">
<per_type_data type="1" atomic_num="14" />
<per_pair_data atnum_i="14" atnum_j="14" AA="7.049556277" BB="0.6022245584"
      p="4" q="0" a="1.80" sigma="2.0951" eps="2.1675" />
<per_triplet_data atnum_c="14" atnum_j="14" atnum_k="14"
      lambda="21.0" gamma="1.20" eps="2.1675" />
</SW_params>
""") # call into Fortran code
si.set_calculator(sw_pot)
e_bulk_per_atom = si.get_potential_energy()/len(si)

# call general purpose elastic constants calculator 
#   using ASE Atoms and QUIP Potential
from matscipy.elasticity import fit_elastic_constants
Cij = fit_elastic_constants(si, optimizer=LBFGS,
                            symmetry='cubic', logfile='-')
vac1 = si.copy()
vac1 *= (3, 3, 3)
half_cell = np.diag(vac1.cell)/2.
vac_atom = ((vac1.positions - half_cell)**2).sum(axis=1).argmin()
del vac1[vac_atom]

vac1.set_calculator(sw_pot)
vac1.rattle(0.01)

opt = LBFGS(vac1)
opt.run(fmax=1e-6)
e_vac = vac1.get_potential_energy() - e_bulk_per_atom*len(vac1)
print('SW vacancy formation energy', e_vac, 'eV')

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 16:55:25      -34.635777*       0.3247
LBFGS:    1 16:55:25      -34.644590*       0.1504
LBFGS:    2 16:55:25      -34.646997*       0.0001
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 16:55:25      -34.670667*       0.1584
LBFGS:    1 16:55:25      -34.672777*       0.0749
LBFGS:    2 16:55:25      -34.673385*       0.0000
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 16:55:25      -34.678737*       0.0000
       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 16:55:25      -34.660845*       0.1508
LBFGS:    1 16:55:25      -34.662784*       0.0742
LBFGS:    2 16:55:25      -34.663403*       0.0000
       Step     Time          Energy         fmax
*Force-consistent energies used in optimizat